In [13]:
import re
import pandas as pd

In [14]:
re_epoch_msg = re.compile(
'Epoch (?P<epoch>\d+),  (?P<start_ts>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) - (?P<end_ts>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})\n'  # epoch & time stamp
'Last reward: ((?P<last_reward>[-]*\d+)|(None))\n'  # reward
'  TrafficEmulator.generate_traffic\(\): located (?P<session_in>\d+), droped (?P<session_out>\d+), left (?P<session_net>\d+) sessions in epoch \d+.\n'  # session dynamics
'  TrafficEmulator.generate_requests_\(\): generated (?P<req_generated>\d+) requests at epoch \d+.\n'  # how many requests generated?
'Observation: \((?P<ob_last_q>\d+), (?P<ob_last_t>\d+), (?P<ob_new_q>\d+)\)\n'  # current observation slice
'(    QAgentNN:  batch action distribution: (?P<batch_dist>\{\(False, \'serve_all\'\): (?P<batch_dist_wake>[a-zA-Z\d\.-]+), \(True, None\): (?P<batch_dist_sleep>[a-zA-Z\d\.-]+)\})\n){0,1}'  # batch distribution
'  QAgentNN:  '
    '(?P<agent_update_msg>(last_reward is None, agent not updated.)|'
    '(state is None, agent not updated.)|(unfull memory.)|'
    '(frozen net at counter (?P<counter>\d+).)|(update loss is (?P<loss>[\d\.]+) at counter \d+, reward_scaling is (?P<reward_scaling>[a-zA-Z\d\.-]+))'
    ')\n'  # agent update message: reason, counter, & loss
'  QAgent:  (?P<agent_act_msg>'
             '((randomly choose action( \([a-zA-Z,_ \']+\)){0,1})|(choose best q among (?P<q_vals>\{\(False, \'serve_all\'\): (?P<q_wake>[a-zA-Z\d\.-]+), \(True, None\): (?P<q_sleep>[a-zA-Z\d\.-]+)\})))'
             ' \((?P<agent_act_basis>[a-zA-Z ]+)\).)\n'
'Control: (?P<agent_action>\([a-zA-Z,_ \']+\)), Agent update: [a-zA-Z\d\.-]+\n'
'(  TrafficServer: Serving all \d+ requests in queue.\n    TrafficServer: Dropped \d+ q entries to deduplicate\n  TrafficServer:  \(False, \'serve_all\'\)\n){0,1}'
'  TrafficEmulator.evaluate_service_\(\): '
    'served (?P<req_served>\d+), queued (?P<req_queued>\d+), '
    'rejected (?P<req_rejected>\d+) \((?P<req_retried>\d+), (?P<req_canceled>\d+)\), unattended (?P<req_unattended>\d+) at epoch \d+, '
    'reward [-]*\d+ \((?P<tr_reward_serve>[-]*\d+), (?P<tr_reward_wait>[-]*\d+), (?P<tr_reward_fail>[-]*\d+)\)\n'
'  TrafficEmulator.evaluate_service_\(\): buffer info: pending (?P<req_pending_all>\d+), waiting (?P<req_waiting_all>\d+), served (?P<req_served_all>\d+), failed (?P<req_failed_all>\d+)\n'
'Cost: (?P<op_cost>[-]*\d+), Reward: (?P<tr_reward>[-]*\d+)\n{0,1}'
)

In [15]:
def index_file(file):
    with open(file, "r") as f_log:
        all_log = "".join(f_log.readlines()).split('\n\n')
        extract = [re_epoch_msg.search(piece) for piece in all_log]
        df = pd.DataFrame.from_dict([piece.groupdict() for piece in extract if piece is not None])
        df.set_index('epoch')
        df.index.name = 'epoch'
    with open("index_"+file+".csv", "w") as f_ind:
        df.to_csv(f_ind, sep=';', index=True, header=True)

In [20]:
files = ["message_2016-6-4_1641.log",
 ]
map(index_file, files)

IOError: [Errno 2] No such file or directory: 'message_2016-6-4_1641.log'